<div style="background-color: #ffffff; color: #000000; padding: 10px;">
<img src="../media/img/kisz_logo.png" width="192" height="69"> 
<h1> Working with embeddings:
<h2>An introductory workshop with applications on Semantic Search
</div>

<div style="background-color: #f6a800; color: #ffffff; padding: 10px;">
<h2>Part 3.1 - Word2Vec
</div>

In this section we will take a look to the Word2Vec family and its two different approaches. We will use <kbd>gensim</kbd> for training from scratch a model and we'll compare it with a pretrained model, using different tools and utilities included in <kbd>gensim</kbd>. Back to our use case, we will see how our movie search engine performs with Word2Vec.

We start as usual importing some packages.

In [ ]:
# imports
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

from nb_config import INTERIM_DATA_PATH

from src.data import data_loader

import warnings
warnings.filterwarnings('ignore')

<div style="background-color: #dd6108; color: #ffffff; padding: 10px;">
<h3>1. Overview
</div>

Word2Vec is a popular word embedding model that captures semantic relationships between words in a corpus. Unlike traditional methods such as BoW or TF-IDF, Word2Vec embeds words into continuous vector spaces, preserving their semantic meanings and relationships.

To create a Word2Vec model, two main architectures are commonly employed, Continuous Bag of Words (CBOW) and Skip-gram. Both approaches involve training a neural network on a large dataset, typically comprising sentences or documents, to learn word embeddings. These embeddings are vectors that position words in a multi-dimensional space based on their contextual similarities.
- In the CBOW architecture, the model predicts the target word given its context, or the surrounding words.
- The Skip-gram architecture predicts the context words based on a given target word.

Training the Word2Vec model involves adjusting the vector representations of words to minimize the difference between predicted and actual words. Once the Word2Vec model is trained, the resulting word vectors can be used to measure semantic relationships between words. Words with similar meanings or contexts will have vectors that are close together in the vector space. This allows for semantic operations, such as vector subtractions, where the vector for "king" minus "man" plus "woman" may yield a vector close to "queen."

We are going to need at this point our tokens. Let's load them.

In [ ]:
df, params = data_loader("my_tokenized_data.parquet")

# extract the arguments for the normalize function
tokenizer = params['tokenizer']
arguments = params['args']

# print the tokenizer used getting the tokens
print(f"Tokenizer used: {tokenizer}")
print(f"Arguments passed to the normalize function:\n{arguments}")


<div style="background-color: #dd6108; color: #ffffff; padding: 10px;">
<h3>2. Training your own word2vec embeddings with <kbd>gensim</kbd>
</div>

Let's train a Word2Vec model with our data. We will just use our tokenized descriptors as training data.

In [ ]:
from gensim.models import Word2Vec

# pack the tokens as a list of lists
tokens = df.tokens.apply(lambda x: x.tolist()).tolist()

# instantiate the model with our data as training corpus
model = Word2Vec(sentences=tokens, sg=1, vector_size=300, window=5, min_count=1, workers=4)

# proveide fast access to the vectors
w2v_1 = model.wv

And voilà! We have created a Word2Vec model trained from us.

<div style="background-color: #dd6108; color: #ffffff; padding: 10px;">
<h3>3. Using pretrained word2vec embeddings with <kbd>gensim</kbd>
</div>

We can also used pre-trained embeddings. They have been trained on large and diverse datasets, so they can usually capture general semantic relationships and linguistic patterns better than our generated embeddings. Using pretrained embeddings also saves resources and time, as you can leverage embeddings that have already been trained on powerful hardware and large-scale datasets.

We will start taking a look to the different embeddings that gensim can offer.

In [ ]:
import gensim.downloader as api

# Show all available models in gensim-data
print(*list(api.info()['models'].keys()), sep = "\n")

We will load the <kbd>word2vec-google-news-300</kbd> embeddings. This model was trained on a part of the Google News dataset (about 100 billion words) and contains 300-dimensional vectors for 3 million words and phrases. The phrases were obtained using a simple data-driven approach described in the paper ['Distributed Representations of Words and Phrases and their Compositionality'](https://arxiv.org/abs/1310.4546).


> <details>
> <summary>Too small for you?</summary>
> You can also try the [word2vec model](https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit?usp=sharing) released by Google. It’s 1.5GB and includes word vectors for a vocabulary of 3 million words and phrases that they trained on roughly 100 billion words from a Google News dataset.
>
> You can even load this model with gensim. Download it, unzip it the <kbd>data/external</kbd> folder and use following code to get started:
> 
> <pre><code>from gensim.models.keyedvectors import KeyedVectors
> # load the model
>w2v_3 = KeyedVectors.load_word2vec_format(path_to_model, binary=True)
> </code></pre>
> 
> </details>

In [ ]:
w2v_2 = api.load('word2vec-google-news-300')

<div style="background-color: #dd6108; color: #ffffff; padding: 10px;">
<h3>4. Working with the embeddings
</div>

The embeddings in the <kbd>w2v_1</kbd> and <kbd>w2v_2</kbd> objects can be accessed exactly as if they were a dictionary, using as keys the different tokens.

```
        w2v_1['<token>']
```

Working with Gensim has some benefits. This libary provides a lot of useful methods to work with these embeddings. Let's see some of them!

<div style="background-color: #b1063a; color: #ffffff; padding: 10px;">
<strong>Exercise</strong>

You will find in the following cells different functionalities of <kbd>gensim</kbd>. Check them for your two models:
- **<kbd>w2v_1</kbd>**, trained on our data
- **<kbd>w2v_2</kbd>**,  loaded from pre-trained embeddings

You can also change the examples or tokens used for each functionality to get a better understanding of their strenghts and limitations. 

How are the results for both models? What tasks is each model good at?
</div>


1. **Cosine Similarity**: We can directly measure the (cosine) similarity between two tokens 

In [ ]:
# tokens to compare
compared_tokens = ['woman', 'man']

# calculating the cosine similarity
similarity = w2v_1.similarity(*compared_tokens)
print(f"Similarity of {compared_tokens} is {similarity:.6f}")


2. **Finding similar tokens**: We can look for the most similar tokens, given a single token as reference. The result provides a list with the 10 closest tokens. We will show only the first one.

In [ ]:
result = w2v_1.similar_by_word("cat")
print("{}: {:.6f}".format(*result[0]))



3. **Word Analogies**:  We can look for the most similar tokens, given not only one but several tokens as reference, in a way that can be used for analogy questions. As example, we may want to know what would be for *woman*, what *king* is for *man*. We can use the <kbd>most_similar</kbd> method. As before, the result provides a list with the 10 closest tokens and we will show only the first one.

In [ ]:
positive_tokens = ['woman', 'king']
negative_tokens = ['man']

result = w2v_1.most_similar(positive=positive_tokens, negative=negative_tokens)
print("{}: {:.6f}".format(*result[0]))


4. **The odd one out**: We can compare tokens to see which one is less like the others. For example, if we provide the tokens *cat*, *dog*, *horse*, *fridge* and *cow*, we would expect the token *fridge* to be more different than the rest.

In [ ]:
match_list = ['cat', 'dog', 'horse', 'fridge', 'cow']
result = w2v_1.doesnt_match(match_list)
print(f"In the list {match_list} the token '{result}' doesn't match")


5. **Sentence comparison**: We can compare sentences. For that we tokenize the words of each sentence and pass the tokens lists to the model as shown here:

In [ ]:
sentence_1 = "Peter's parents went with him for a walk".lower().split()

sentence_2 = "The couple went with their son Peter for a walk".lower().split()

sim = w2v_1.n_similarity(sentence_1, sentence_2)
print("{:.6f}".format(sim))

<div style="background-color: #b1063a; color: #ffffff; padding: 10px;">
<strong>Exercise (optional)</strong>

Did you had any problem with tokens that were not in the vocabulary? How did <kbd>gensim</kbd> handle it? How would you handle it?
</div>

<div style="background-color: #dd6108; color: #ffffff; padding: 10px;">
<h3>5. Queries with Word2Vec
</div>

We will come back now to our original task: building a Semantic Search Engine. We are going to check now how our new models perform with that task.

> **NOTE**: As the model with pre-trained embeddings performs much better than the one we trained with our data, we will use <kbd>w2v_2</kbd> as standard. Feel free to try the queries also with the other model if you want.

But... Word2Vec produces word embeddings and not sentence or text embeddings. How can we get embeddings for our descriptors? We have many different options, but at this point the most common solutions would be:

>- **Average of Word2Vec vectors** : You can just take the average of all the word vectors in a sentence. This average vector will represent your sentence vector. That's the usual approach and the one will be doing here.
>- **Average of Word2Vec vectors with TF-IDF**: Using TF-IDF with this Word2Vec embeddings we could leverage the importance and thus the weight of the different tokens. We will leave that as a homework by now. 
>- **Doc2Vec** : you can train your dataset using Doc2Vec and then use the sentence vectors. We will actually be doing this in the **Notebook 3.4.- Doc2Vec**, so we don't have to worry about it by now.

<div style="background-color: #b1063a; color: #ffffff; padding: 10px;">
<strong>Exercise</strong>

Create the embeddings for the descriptors in our dataset, by averaging the vectors for the tokens in each descriptor. Present the result as a matrix with so many rows as movies)and so many columns as dimensions in our embeddings, and store it in a variable called <kbd>corpus_matrix</kbd>.

Make sure that out of dictionary tokens don't break the code.
</div>  
<br>

> <details>
> <summary>Need some help?</summary>
> 
> Take a look to the [API](https://radimrehurek.com/gensim/apiref.html) of Word2Vec models by <kbd>gensim</kbd>. It could also help knowing that in this library Word2Vec models are a type of *keyedvectors*...
> </details>
<br>

> <details>
> <summary>Still clueless?</summary>
>
> Here is some code that could do the trick:
>
> <pre><code># creates a Pandas series with the averaged vectors
> corpus_matrix = df.loc[:, 'tokens'].map(lambda x: w2v_2.get_mean_vector(x, ignore_missing=True))
> # creates the matrix from the vectors
> corpus_matrix = np.vstack(corpus_matrix)
> </code></pre>
> 
> </details>

In [ ]:
# creates a Pandas series with the averaged vectors
corpus_matrix = ...
# creates the matrix from the vectors
corpus_matrix = ...

How does it look like?

In [ ]:
print(f"Shape of our corpus matrix: {corpus_matrix.shape}\n")
corpus_matrix

<div style="background-color: #b1063a; color: #ffffff; padding: 10px;">
<strong>Exercise</strong>

Create the embedding of the query, and find its distance to the other vectors following the next steps:
1. Tokenize the query with the right tokenizer and parameters
2. Calculate the averaged embedding from the token vectors
3. Reshape the query vector to turn it into a 2-dimensional array, by adding an extra dimension behind
4. Calculate the euclidean distance, dot product similarity and cosine similarity as we have done before
</div>  
<br>

> <details>
> <summary>If you get lost...</summary>
> 
> We don't think you should have problems with steps 1. and 4. as you have done it already several times in the past notebooks, but for the second and third step, that culd help...
>
> <pre><code># creating the averaged vector
> query_vector = w2v_2.get_mean_vector(query_tokens, ignore_missing=True)
> # reshaping the array 
> query_vector = query_vector.reshape((1, -1))
> </code></pre>
> </details>

In [ ]:
query = "An innocent man in prison that never loses the hope starts helping the warden as accountant"

# code for tokenizing the query

query_tokens = ...

In [ ]:
# code for calculating the embeddings
query_vector = ...

# code for reshaping the array
query_vector = ...


In [ ]:
# calculating the distances
euclid_distances = ...
dotprod_similarities = ...
cos_similarities = ...

Now we are going to create a dataframe to store all those values, as we did in the last notebooks.

In [ ]:
# creating a new dataframe and adding the extra columns
results = df.loc[:, ['title', 'descriptor']].copy()
results.loc[:, 'euclid_dist'] = euclid_distances
results.loc[:, 'dot_prod_sim'] = dotprod_similarities
results.loc[:, 'cos_sim'] = cos_similarities
results.loc[:, 'common_tokens'] = df.loc[:, 'tokens'].map(lambda x: list(set(x).intersection(query_tokens)))

<div style="background-color: #b1063a; color: #ffffff; padding: 10px;">
<strong>Exercise</strong>

Which metrics show better results? You can try several different queries with varying degrees of complexity.
</div>

In [ ]:
# choose the metric! options:
# 'euclid_dist', 'dot_prod_sim', 'cos_sim'
metric = 'cos_sim'

# choose number of results
n = 10

# show the results
results.sort_values(by=metric, ascending=False).head(n)

<div style="background-color: #b1063a; color: #ffffff; padding: 10px;">
<strong>Exercise (optional)</strong>

How do tokens influence the final result? Could we achieve better results with other type of tokens?

Try to create tokens in a different way as explained in **Notebook 2.1.- Text normalization** (different tokenizer, with or without lemmatizing, stemming, stop words, case folding,...) and check the performance again.

Does <kbd>gensim</kbd> provide also any tokenizer?
</div>

<br>

<div style="background-color: #b1063a; color: #ffffff; padding: 10px;">
<strong>Exercise (optional)</strong>

Would it be different if instead of using the average of the vectors in the descriptor we used the average fo the vectors weighted with TF-IDF?

Write an implementation of TF-IDF for Word2Vec embeddings and try it with our task. How does it perform compared to the raw average?
</div>

<div style="background-color: #dd6108; color: #ffffff; padding: 10px;">
<h3>6. Advantages and disadvantages of Word2Vec Models
</div>

Let's summarize the pros and cons of Word2Vec embeddings, and see where can they be used.
        
#### Advantages:

> **Semantic Relationships**: Word2Vec models capture relationships between words by representing them as dense vectors in a continuous vector space. While this representation allows for measuring global semantic similarities, it is more static compared to contextual embeddings. The semantic relationships in Word2Vec are derived from the overall usage patterns of words across the entire training corpus.
> - **Word Similarity and Analogies**: Word2Vec models excel in capturing word similarities and analogies. By embedding words in a vector space, the model can perform operations such as vector arithmetic, allowing for analogical reasoning (e.g., king - man + woman = queen). This ability to represent and understand relationships between words is a significant strength.
>  - **Contextual Information**: Word2Vec considers the context in which words appear, capturing not only the word itself but also the surrounding context. This contextual information contributes to a richer and more meaningful representation of words, addressing the limitations of BoW and TF-IDF models in terms of word order and semantics.
>  - **Dimensionality Reduction**: Word2Vec models often result in lower-dimensional representations compared to high-dimensional BoW and TF-IDF vectors. This reduction in dimensionality contributes to computational efficiency and facilitates more effective use of the learned embeddings.

#### Disadvantages:

> - **Lack of Interpretability**: While Word2Vec models provide powerful representations, the dense vector embeddings lack interpretability. It can be challenging to understand the specific factors contributing to the position of a word in the vector space, making it less transparent compared to BoW or TF-IDF models.
> - **Dependency on Training Data Quality**: The quality of Word2Vec embeddings heavily depends on the training data. If the training corpus is not representative or lacks diversity, the model may produce biased or limited embeddings. Additionally, rare or out-of-vocabulary words may not be well-represented if not encountered frequently during training.
> - **Fixed Vocabulary Size**: Word2Vec models typically have a fixed vocabulary size determined during training. This can be a limitation when dealing with evolving or dynamic vocabularies, as the model may struggle to adapt to new words introduced after training.
> - **Difficulty Handling Polysemy**: Word2Vec models may struggle with polysemy, where a single word has multiple meanings. Since the model assigns a single vector to each word, it might not effectively capture the different senses of polysemous words.

#### Applications:

> - **Semantic Similarity and Clustering**: Word2Vec models find applications in measuring semantic similarity between words and clustering words with similar meanings. This is valuable in various natural language processing tasks, such as document similarity, information retrieval, and recommendation systems.
>  - **Named Entity Recognition (NER)**: In Named Entity Recognition tasks, where the goal is to identify and classify entities (e.g., names of people, organizations) in text, Word2Vec embeddings can enhance the contextual understanding of words, improving the accuracy of entity recognition.
>  - **Sentiment Analysis**: Word2Vec embeddings contribute to sentiment analysis tasks by capturing nuanced relationships between words and their emotional connotations. This enables more accurate sentiment classification in comparison to simpler models like BoW.
> - **Language Translation**: Word2Vec embeddings have been utilized in machine translation tasks to capture cross-lingual semantic similarities, facilitating the translation of words and phrases based on their contextual representations.